In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/jeroen/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [13]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [23]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [28]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [33]:
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [36]:
padded.shape

(25000, 120)

In [34]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

b'just because someone is under the age of 10 does not mean they are stupid if your child likes this film you 'd better have him her tested i am continually amazed at how so many people can be involved in something that turns out so bad this film is a showcase for digital <OOV> and nothing else the writing is horrid i can 't remember when i 've heard such bad dialogue the songs are beyond wretched the acting is sub par but then the actors were not given much who decided to employ joey <OOV> he cannot sing and he is ugly as sin br br the worst thing is the <OOV> of it all it is as
b'Just because someone is under the age of 10 does not mean they are stupid. If your child likes this film you\'d better have him/her tested. I am continually amazed at how so many people can be involved in something that turns out so bad. This "film" is a showcase for digital wizardry AND NOTHING ELSE. The writing is horrid. I can\'t remember when I\'ve heard such bad dialogue. The songs are beyond wretched. 

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.5130 - accuracy: 0.7227 - val_loss: 0.3555 - val_accuracy: 0.8427
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2435 - accuracy: 0.9045 - val_loss: 0.3730 - val_accuracy: 0.8386
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0895 - accuracy: 0.9774 - val_loss: 0.4545 - val_accuracy: 0.8261
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0224 - accuracy: 0.9974 - val_loss: 0.5193 - val_accuracy: 0.8272
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0069 - accuracy: 0.9994 - val_loss: 0.5808 - val_accuracy: 0.8273
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.6335 - val_accuracy: 0.8267
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 9.2684e-04 - accuracy: 1.0000 - val_loss: 0.6768 - val_accuracy: 0.8274
Ep

In [9]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [10]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [11]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [12]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1429], [969], [4], [1538], [1538], [4739], [], [790], [2012], [11], [2920], [2190], [], [790], [2012], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2920], [1276], [], [], [2012], [1005], [2920], [969], [579], [790], []]
